In [26]:
audio_input = ''
language = ''

In [ ]:
  !pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0\
  InstructorEmbedding

In [27]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
import timeit
import sys

In [28]:
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','c9be9891-57c9-4ace-a24b-47d7daea138a')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [29]:
loader = DirectoryLoader('', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

print(documents)

[Document(page_content=' \n \n \n \n  \nOPERATING MANUAL \nFOR \nINDIAN RAILWAYS \n \nGOVERNMENT OF INDIA \nMINISTRY OF RAILWAYS \n(RAILWAY BOARD) \n \n', metadata={'source': 'operating manual-traffic.pdf', 'page': 0}), Document(page_content=' \n \n \n \n \n \n \n \n  \n \n  \n \n \n ', metadata={'source': 'operating manual-traffic.pdf', 'page': 1}), Document(page_content='INDEX \n \nS.No. Chapters Page No. \n1 Working of Stations   1-12  \n2 Working of Trains  \n 13-14 \n3 Marshalling  \n 15-18 \n4 Freight Operation  \n 19-27 \n5 Preferential Schedule & Rationalization Order  \n 28 \n6 Movement of ODC and Other Bulky Consignment  \n 29-32 \n7 Control Organisation  \n 33-44 \n8 Command, Control & Coordination of Emergency Rescue Operations on the \nopen line  \n 45-54 \n9 Marshalling Yards and Freight Terminals  \n 55-67 \n10 Container Train Operation  \n 68-72 \n11 Customer Interface and Role of Commercial Staff  \n 73-77 \n12 Inspections  \n 78-83 \n13 Interlocking  84-92 \n14 Statio

In [30]:
len(documents)

169

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
docs = text_splitter.split_documents(documents)
len(docs)

1195

In [32]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [33]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="railway-chatbot"

In [34]:
#docsearch=Pinecone.from_existing_index(index_name, embeddings)
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [35]:
query = "what are the Objectives of incident Management Plan"
docs=docsearch.similarity_search(query, k=3)
print(docs)

[Document(page_content='iii) Particular note should be taken of the need to provide suitable training to keep staff aware of what \nneeds to be done in the event of a serious incident. \n \n1. Objectives of incident Management Plan  \nIn Order of priority these are:- \n \ni) Save lives and alleviate suffering \nii) Provide help to stranded passengers and  arrange their prompt evacuation \niii) Instil a sense of security amongst all c oncerned by providing accurate information \niv) Protect Railway property', metadata={}), Document(page_content='iii) Particular note should be taken of the need to provide suitable training to keep staff aware of what \nneeds to be done in the event of a serious incident. \n \n1. Objectives of incident Management Plan  \nIn Order of priority these are:- \n \ni) Save lives and alleviate suffering \nii) Provide help to stranded passengers and  arrange their prompt evacuation \niii) Instil a sense of security amongst all c oncerned by providing accurate info

In [36]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that I don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [37]:
chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [38]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 2}),return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [1]:
#ASR
import speech_recognition as sr

import sys

#sys.stdin.reconfigure(encoding='utf-8')
#sys.stdout.reconfigure(encoding='utf-8')

# Initialize the recognizer
recognizer = sr.Recognizer()

# Use a microphone as the audio source
with sr.Microphone() as source:
    print("Listening...")
    
    # Adjust for ambient noise
    recognizer.adjust_for_ambient_noise(source)
    
    # Listen to the user's speech
    audio = recognizer.listen(source)

    print("Processing...")

try:
    # Recognize speech in Kannada
    language = "kannada"
    if language=="kannada":
        lang="kn-IN"
    elif language=="hindi":
        lang="hi-IN"
    else:
        lang="en"
    text = recognizer.recognize_google(audio, language=lang)

    # Print the recognized text
    print("you said:", text)

except sr.UnknownValueError:
    print("Error occured")

except sr.RequestError as e:
    print(f"Error occured; {e}")

Listening...


KeyboardInterrupt: 

In [69]:
import googletrans
from googletrans import Translator

trans = Translator()

if lang=="kn-IN":
    lang="kn"

elif lang=="hi-IN":
    lang="hi"

if lang=="kn":
    target_lang = "en"
    text = trans.translate(text, src=lang, dest=target_lang).text

elif lang=="hi":
    target_lang = "en"
    text = trans.translate(text, src=lang, dest=target_lang).text

print(text)

What is the purpose of vehicle plans


In [70]:
#user_input=input(f"Input Prompt:")
user_input = text
if user_input=='exit':
    print('Exiting')
    sys.exit()
result=qa({"query": user_input})
print("Response : ", result["result"])

PineconeProtocolError: Failed to connect; did you specify the correct index name?

In [61]:
text = trans.translate(result["result"], src="en", dest=lang).text

print(text)

ಘಟನೆ ನಿರ್ವಹಣಾ ಯೋಜನೆಯ ನಿರ್ವಹಣಾ ಯೋಜನೆ ಉದ್ದೇಶಗಳು ಜೀವಗಳನ್ನು ಉಳಿಸುವುದು, ದುಃಖವನ್ನು ನಿವಾರಿಸುವುದು, ಸಿಕ್ಕಿಬಿದ್ದ ಪ್ರಯಾಣಿಕರಿಗೆ ಸಹಾಯ ನೀಡುವುದು, ಅವರ ತ್ವರಿತ ಸ್ಥಳಾಂತರಿಸುವಿಕೆಯನ್ನು ವ್ಯವಸ್ಥೆ ಮಾಡುವುದು, ನಿಖರವಾದ ಮಾಹಿತಿಯನ್ನು ಒದಗಿಸುವ ಮೂಲಕ ಸಂಬಂಧಪಟ್ಟ ಎಲ್ಲರ ನಡುವೆ ಭದ್ರತೆಯ ಪ್ರಜ್ಞೆಯನ್ನು ತುಂಬುವುದು ಮತ್ತು ರೈಲ್ವೆ ಆಸ್ತಿಯನ್ನು ರಕ್ಷಿಸುವುದು.


In [62]:
import pyttsx3
engine = pyttsx3.init()

engine.say(text)
engine.runAndWait()